In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('./data/Cannabis_Strains_Features_cannabis.csv')

In [26]:
print(df.shape, '\n')
df.head(10)

(2316, 7) 



,strain_id,strain,type,Rating,effects,description,Flavors
1838,1838,silver nina,hybrid,5.0,"creative, energetic, focused, happy, aroused",silver nina by colorado seed inc. is the stabi...,citrus
1834,1834,silver calyx,hybrid,5.0,"aroused, energetic, euphoric, focused, relaxed",silver calyx by calyx garden is a balanced hyb...,"lemon, tree, fruit, pine"
1779,1779,royal tree sherbet,hybrid,5.0,relaxed,sherbet by royal tree gardens is another genet...,"tropical, sweet, berry"
1804,1804,scouts honor,hybrid,5.0,"relaxed, creative, giggly, talkative, uplifted",scout’s honor is an indica-dominant hybrid cre...,"sweet, tropical, flowery"
1805,1805,scroopy noopers,hybrid,5.0,"euphoric, hungry, relaxed",scroopy noopers by green dream is a modern hom...,"pungent, coffee"
262,262,blood orange kush,indica,5.0,"sleepy, uplifted, euphoric, happy, creative",blood orange kush is an indica-dominant cross ...,"diesel, citrus, orange"
1809,1809,secret garden og,indica,5.0,"euphoric, talkative, uplifted, creative, energ...",secret garden og is a colorado seed inc. maste...,"flowery, lavender, honey"
258,258,blizzard bush,hybrid,5.0,"talkative, uplifted, creative, energetic, euph...",blizzard bush by greenpoint seeds is a resinou...,"flowery, earthy, pungent"
997,997,hawaiian trinity,hybrid,5.0,"talkative, uplifted, creative, euphoric, focused",hawaiian trinity by pua mana 1st hawaiian paka...,"sweet, tropical, flowery"
1835,1835,silver cindy,hybrid,5.0,"aroused, uplifted, energetic, euphoric, happy",colorado seed inc. brings speedy effects and e...,none


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2351 entries, 0 to 2350
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Strain       2351 non-null   object 
 1   Type         2351 non-null   object 
 2   Rating       2351 non-null   float64
 3   Effects      2351 non-null   object 
 4   Flavor       2350 non-null   object 
 5   Description  2350 non-null   object 
dtypes: float64(1), object(5)
memory usage: 110.3+ KB


In [5]:
nans = df.isnull().sum()

In [6]:
nans

Strain         0
Type           0
Rating         0
Effects        0
Flavor         1
Description    1
dtype: int64

In [7]:
# Fixing formatting
df['Effects'] = df['Effects'].str.lower()
df['Flavor'] = df['Flavor'].str.lower()
df['Description'] = df['Description'].str.lower()
df['Strain'] = df['Strain'].str.lower()

In [8]:
#make the data PostgreSQL friendly
df["Description"] = df["Description"].str.replace("'", '')
df.Effects = df.Effects.str.replace(",", ", ")
df.Flavor = df.Flavor.str.replace(",", ", ")
df['Strain'] = df['Strain'].str.replace('-', ' ')
df.replace(np.nan, '', inplace=True)

In [9]:
df['Rating'] = df['Rating'].apply(lambda s: np.NaN if s == 0.0 else s)
df['Effects'] = df['Effects'].apply(lambda s: np.NaN if s == "None" else s)
df['Flavors'] = df['Flavor'].apply(lambda s: np.NaN if s == "None" else s)
df = df.drop(columns=['Flavor'])

In [10]:
df = df.dropna().reset_index().drop(columns=['index']).reset_index()

In [11]:
# sorting the values by the ratings
df = df.sort_values(by=['Rating'], ascending=False)

In [12]:
df['Rating'].min(), df['Rating'].max()

(1.0, 5.0)

In [13]:
def transpose(x):
    return round((x - 2.8) * (5.0 / (5.0 - 2.8)), 2)

In [14]:
df['Rating'] = df['Rating'].apply(transpose)

In [15]:
# replace index column name to Strain_ID
df = df.rename(columns = {'index': 'Strain_ID'})


In [16]:
# df = df.replace({'': 'index'})

In [17]:
# changing headers from upper to lower case
df = df.rename(columns = {'Strain_ID': 'strain_id', 'Strain': 'strain', 'Type': 'type', 'Effects': 'effects', 'Flavor': 'flavor',
                         'Description': 'description'})

In [18]:
df.head(10)

,strain_id,strain,type,Rating,effects,description,Flavors
1838,1838,silver nina,hybrid,5.0,"creative, energetic, focused, happy, aroused",silver nina by colorado seed inc. is the stabi...,citrus
1834,1834,silver calyx,hybrid,5.0,"aroused, energetic, euphoric, focused, relaxed",silver calyx by calyx garden is a balanced hyb...,"lemon, tree, fruit, pine"
1779,1779,royal tree sherbet,hybrid,5.0,relaxed,sherbet by royal tree gardens is another genet...,"tropical, sweet, berry"
1804,1804,scouts honor,hybrid,5.0,"relaxed, creative, giggly, talkative, uplifted",scout’s honor is an indica-dominant hybrid cre...,"sweet, tropical, flowery"
1805,1805,scroopy noopers,hybrid,5.0,"euphoric, hungry, relaxed",scroopy noopers by green dream is a modern hom...,"pungent, coffee"
262,262,blood orange kush,indica,5.0,"sleepy, uplifted, euphoric, happy, creative",blood orange kush is an indica-dominant cross ...,"diesel, citrus, orange"
1809,1809,secret garden og,indica,5.0,"euphoric, talkative, uplifted, creative, energ...",secret garden og is a colorado seed inc. maste...,"flowery, lavender, honey"
258,258,blizzard bush,hybrid,5.0,"talkative, uplifted, creative, energetic, euph...",blizzard bush by greenpoint seeds is a resinou...,"flowery, earthy, pungent"
997,997,hawaiian trinity,hybrid,5.0,"talkative, uplifted, creative, euphoric, focused",hawaiian trinity by pua mana 1st hawaiian paka...,"sweet, tropical, flowery"
1835,1835,silver cindy,hybrid,5.0,"aroused, uplifted, energetic, euphoric, happy",colorado seed inc. brings speedy effects and e...,none


In [19]:
# checking the ratings values
df['Rating'].min(), df['Rating'].max(), len(df['Rating'])

(-4.09, 5.0, 2316)

In [20]:
# cleaning using RegEx re.sub() function to replace '\n' sub-string with "".
df['effects'] = df['effects'].apply(lambda x : re.sub(r'\n', "", x))
df['effects']

1838         creative, energetic, focused, happy, aroused
1834       aroused, energetic, euphoric, focused, relaxed
1779                                              relaxed
1804       relaxed, creative, giggly, talkative, uplifted
1805                            euphoric, hungry, relaxed
                              ...                        
1520     uplifted, creative, energetic, euphoric, focused
2048                                       happy, relaxed
269             giggly, uplifted, creative, happy, sleepy
2274    talkative, uplifted, creative, energetic, euph...
1698                                                 none
Name: effects, Length: 2316, dtype: object

In [21]:
df.head()

,strain_id,strain,type,Rating,effects,description,Flavors
1838,1838,silver nina,hybrid,5.0,"creative, energetic, focused, happy, aroused",silver nina by colorado seed inc. is the stabi...,citrus
1834,1834,silver calyx,hybrid,5.0,"aroused, energetic, euphoric, focused, relaxed",silver calyx by calyx garden is a balanced hyb...,"lemon, tree, fruit, pine"
1779,1779,royal tree sherbet,hybrid,5.0,relaxed,sherbet by royal tree gardens is another genet...,"tropical, sweet, berry"
1804,1804,scouts honor,hybrid,5.0,"relaxed, creative, giggly, talkative, uplifted",scout’s honor is an indica-dominant hybrid cre...,"sweet, tropical, flowery"
1805,1805,scroopy noopers,hybrid,5.0,"euphoric, hungry, relaxed",scroopy noopers by green dream is a modern hom...,"pungent, coffee"


In [22]:
df.to_csv('cannabis_strain.csv')